In [1]:
VERSION = '21_v2_14'
FOLD_PATH = '../../00_EDA/00_v2_05/result/'
#MODEL = 'microsoft/deberta-v3-large'
GENERATOR = 'microsoft/deberta-xlarge'
#'microsoft/deberta-v3-large'
#'microsoft/deberta-large'
#'anferico/bert-for-patents'
#'microsoft/deberta-v3-base'
#'microsoft/cocolm-large'
LR = 6e-6 #8e-6
HEAD_LR = 6e-6 #8e-6
SEED = 2022
TRN_BS = 1
VAL_BS = 1
ACCUM_STEP = 1
EPOCHS = 20 #10
HIDDEN_DROP_PROB = 0
P_DROP = 0.5
WARMUP_RATIO = 0.05
RATIO_MASKING = 0.25
EVAL_STEP = -1
FP16 = 'false'
STOP_EPOCH = EPOCHS
RESTART_EPOCH = 0

SCHEDULER = 'cosine_hard'
NUM_CYCLES = EPOCHS

CP = 'true'

WINDOW_SIZE = -100 #512
INNER_LEN = -100 #384
EDGE_LEN = -100 #64

MAX_LEN = -100 #512

GRAD_CLIP = 1000

PRETRAIN_PATH = 'none' #f'result/{VERSION}/model_gen_seed{SEED}_fold{FOLD}_epoch{RESTART_EPOCH-1}.pth'

In [2]:
FOLD = 0

!python ../$VERSION/pretrain_mlm.py --generator $GENERATOR \
--version $VERSION --fold_path $FOLD_PATH --fold $FOLD --seed $SEED \
--lr $LR --head_lr $HEAD_LR --trn_batch_size $TRN_BS --val_batch_size $VAL_BS \
--epochs $EPOCHS --hidden_drop_prob $HIDDEN_DROP_PROB --p_drop $P_DROP \
--accumulate_grad_batches $ACCUM_STEP --warmup_ratio $WARMUP_RATIO --ratio_masking $RATIO_MASKING \
--eval_step $EVAL_STEP --fp16 $FP16 --stop_epoch $STOP_EPOCH \
--scheduler $SCHEDULER --num_cycles $NUM_CYCLES --check_pointing $CP \
--window_size $WINDOW_SIZE --inner_len $INNER_LEN --edge_len $EDGE_LEN \
--max_length $MAX_LEN --gradient_clip_val $GRAD_CLIP \
--restart_epoch $RESTART_EPOCH --pretrain_path $PRETRAIN_PATH

torch 1.10.2
train_df.shape =  (144293, 8)
load folds...
trn_df.shape =  (115526, 8)
val_df.shape =  (28767, 8)
2022-07-29 22:28:32.318676: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-07-29 22:28:32.318698: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
create validation dataset...
100%|██████████████████████████████████████| 3119/3119 [00:06<00:00, 516.63it/s]
create validation dataset, done
self.warmup_ratio =  0.05
self.num_train_steps =  249500
num_warmup_steps =  12475
lr :  [0.0, 0.0, 0.0]
100%|███████████████████████████████████████| 3119/3119 [10:42<00:00,  4.86it/s]

epoch 1: trn_loss_gen = 6.0629, val_loss_gen = 3.4989, trn_score_gen = 0.3040, val_score_gen = 0.5087
model_gen (best loss) saved


# SWA

In [5]:
!python ../$VERSION/swa_gen.py --work_dir ./result/$VERSION/ --fold 0 --seed $SEED --epochs '16 17 18'

In [6]:
FOLD = 0
WEIGHT_PATH = f'./result/{VERSION}/model_gen_seed{SEED}_fold{FOLD}_swa.pth'

!python ../$VERSION/validation_mlm.py --generator $GENERATOR \
--version $VERSION --fold_path $FOLD_PATH --fold $FOLD --seed $SEED --val_batch_size $VAL_BS \
--weight_path $WEIGHT_PATH \
--window_size $WINDOW_SIZE --inner_len $INNER_LEN --edge_len $EDGE_LEN \
--max_length $MAX_LEN --ratio_masking $RATIO_MASKING

torch 1.10.2
2022-07-31 07:41:55.221351: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-07-31 07:41:55.221372: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
train_df.shape =  (144293, 8)
load folds...
trn_df.shape =  (115526, 8)
val_df.shape =  (28767, 8)
create validation dataset...
100%|██████████████████████████████████████| 3119/3119 [00:05<00:00, 520.95it/s]
create validation dataset, done
100%|███████████████████████████████████████| 3119/3119 [07:30<00:00,  6.93it/s]
val_loss=1.7516, val_score=0.6850
